In [41]:
import numpy as np
import os

In [56]:
def overlapping_read_write(file_gms, seq_file, out_file):
    #work with files
    file = open(file_gms, 'r')
    file_reader = np.loadtxt(file, dtype ='str', skiprows=11, delimiter='\t')
    file.close()
    genes_positions = []
    for gene in file_reader:
        genes_positions.append(gene.split(' '))
    for gene in genes_positions:
        if ('' in gene):
            while('' in gene):
                gene.remove('')
    file_seq = open(seq_file, 'r')
    file_reader = np.loadtxt(file_seq, dtype ='str', skiprows=1, delimiter='\n')
    file_seq.close()
    #regions
    upstream_regions = []
    intersection = []
    sequence = str(file_reader)
    for gene in genes_positions:
        s = []
        strn = ''
        s.append(str(gene[0]))
        if ('<' not in gene[2]) and ('>' not in gene[3]) and (int(gene[2]) > 50) and (int(gene[3]) + 49 < len(sequence) and (int(gene[2]) + int(gene[4]) < len(sequence))):
        
            if gene[1] == '+':
                for i in range(int(gene[2]) - 50, int(gene[2]) - 1):
                    strn += str(sequence[i])
                s.append(strn)
                s.append(int(gene[2]) - 50)
                s.append(int(gene[2]) - 1)
            
            if gene[1] == '-':
                for i in range(int(gene[3]) - 1, int(gene[3]) + 49):
                    strn += str(sequence[i])
                s.append(strn[::-1])
                s.append(int(gene[3]) - 1)
                s.append(int(gene[3]) + 49)
            upstream_regions.append(s)
        elif ('<' not in gene[2]) and ('>' not in gene[3]) : 
            s.append(gene[2])
            s.append(gene[3])
            upstream_regions.append(s)
    
   # print(upstream_regions)
    upstream_regions_complement = []
    for i in range(len(upstream_regions)):
        upstream_region = ''
        for k in upstream_regions[i][1]:
            for j in k:
                if (j == 'T'):
                    upstream_region += 'A'
                if (j == 'A'):
                    upstream_region += 'T'
                if (j == 'C'):
                    upstream_region += 'G'
                if (j == 'G'):
                    upstream_region += 'C'
        upstream_regions_complement.append(upstream_region)
    #overlapping
    overlapping = []
    for i in upstream_regions:
        for j in genes_positions:
            if ('<' not in j[2]) and ('>' not in j[3]):
                if (len(j) == 9 and len(i) == 4):
                  #  print(i, j)
                    if (int(i[3]) > int(j[2]) and i != j and int(i[3]) < int(j[3])):
                        s = [i[0], j[0]]
                        k = [j[0], i[0]]
                        if (s not in overlapping and k not in overlapping):
                            overlapping.append(s)
    #output_writing
    out = open(out_file, 'w')
    out.write('First 100 nucleotides from the upstream regions from each gene:')
    out.write('\n')
    out.write('\n')
    for i in range(len(upstream_regions)):
        if (genes_positions[i][5] != 'atypical'):
            out.write('gene name: <NC_017968>-<GMS2 ')
            out.write(str(upstream_regions[i][0]) + '>')
            out.write('\n')
            out.write(str(upstream_regions[i][1]))
            out.write('\n')
            out.write(str(upstream_regions_complement[i]))
            out.write('\n')
            out.write('\n')
        else:
            out.write('gene name: <NC_017968>-<GMS2 ')
            out.write(str(upstream_regions[i][0]) + '>')
            out.write('\n')
            out.write('NO UPSTREAM REGION FOUND')
            out.write('\n')
            out.write('\n')
    out.write('Numbers of overlapping genes and upstream regions:')
    out.write('\n')
    for i in range(len(overlapping)):
        out.write(str(overlapping[i]))
        out.write('\n')
    out.close()
    return 0

In [57]:
for root, dirs, files in os.walk('/home/emil/Desktop/Science/gms2_viruses_output'):
    for file in files:
        path = os.path.join(root, file)
        s = path.split('/')
        name = s[len(s)-1].replace('_gms2.out','')
        sequence = '/home/emil/Desktop/Science/viruses_genomes/' + name
        outname = '/home/emil/Desktop/Science/overlapping_regions/' + name +'.overlapping.out'
        #print(path)
        overlapping_read_write(path, sequence, outname)